## Alabama State Harvest

In [1]:
# Importing Important libraries
import pandas as pd
from tqdm import tqdm
import re
import time
import urllib
import warnings
from tqdm import trange
import sys
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor

### Test

In [2]:
url="https://arc-sos.state.al.us/cgi/corpdetail.mbr/detail?corp=000000008"

In [3]:
headers={'user-agent':'Mozilla/5.0 (Windows NT 10.0; apple64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'}

In [6]:
res=requests.get(url,headers=headers,verify=False,timeout=10)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'arc-sos.state.al.us'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [7]:
sou=BeautifulSoup(res.content,'lxml')

In [9]:
sou.text.strip()

"Business Entity Records | Alabama Secretary of State\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama Secretary of State\n\n\n\n\n\n\n\nMain navigation\n\n\nSOS Office\n\n\nBiography\n\n\nHistory\n\n\nVoter Registration\n\n\nEvent Scheduling\n\n\nFees & Funds\n\n\nFee Schedule\n\n\nFunds Guide\n\n\nVoter List Fee Schedule\n\n\n\n\nCommittees\n\n\nCommittees Guide\n\n\nMeeting Minutes\n\n\n\n\nStudent Center\n\n\nLegislative Internship\n\n\nSOS Internship Application\n\n\nStudent Poll Worker Internship\n\n\nKids Center\n\n\n\n\nQuarterly & Annual Reports\n\n\n\n\nElections\n\n\nServices\n\n\nAdministrative Services\n\n\nAuthentications\n\n\nTrademarks\n\n\nLands\n\n\n\n\nUCC Services\n\n\nUCC Home\n\n\nOnline Filings\n\n\nFiling Fees\n\n\nProcedures\n\n\nUCC Downloads\n\n\n\n\nBusiness Entities\n\n\nOnline Services\n\n\nDomestic Corporations\n\n\nForeign Corporations\n\n\nHomeowner's Associations\n\n\nLLCs\n\n\nLLPs\n\n\nBusiness Downloads\n\n\n\nAthlete Agents\n\n\nOath of Office Information Center\n

In [10]:
cmp_name=sou.find('thead').find('td',class_="aiSosDetailHead").text.strip()

In [11]:
cmp_name

'A-One Body Shop of Florence, Inc.'

### Function

In [12]:
def harvest_page(num):
  loo_df=pd.DataFrame()
  url=f'https://arc-sos.state.al.us/cgi/corpdetail.mbr/detail?corp={num}'
  headers={ "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,application/signed-exchange;v=b3;q=0.7",
            "Accept-Encoding":"gzip,deflate,br",
            "Accept-Language":"en-US,en;q=0.9,ta;q=0.8",
            "Connection":"keep-alive",
            "Cookie":"_ga=GA1.1.2085200574.1702026901;_ga_HEFYBZ2QN2=GS1.1.1702285595.5.1.1702285603.0.0.0",
            "Host":"arc-sos.state.al.us",
            "Sec-Fetch-Dest":"document",
            "Sec-Fetch-Mode":"navigate",
            "Sec-Fetch-Site":"none",
            "Sec-Fetch-User":"?1",
            "Upgrade-Insecure-Requests":"1",
            "User-Agent":"Mozilla/5.0(WindowsNT10.0;Win64;x64)AppleWebKit/537.36(KHTML,likeGecko)Chrome/120.0.0.0Safari/537.36",
            "sec-ch-ua":"'Not_ABrand';v='8','Chromium';v='120','GoogleChrome';v='120'",
            "sec-ch-ua-mobile":"?0",
            "sec-ch-ua-platform":"Windows"
          }
  try:
    res=requests.get(url,headers=headers,timeout=40)
    sou=BeautifulSoup(res.content,'lxml')
    data_find=sou.text.strip()
    try:
        cmp_name=sou.find('thead').find('td',class_="aiSosDetailHead").text.strip()
        stus_code=res.status_code
        datas0=[]
        datas1=[]

        for i in sou.find('div',attrs={'align':'center'}).find_all('tr'):
          try:
            attri_name=i.find('td',class_="aiSosDetailDesc").text.strip()
            detail_value=i.find('td',class_="aiSosDetailValue").text.strip()

            datas1.append(detail_value)
            datas0.append(attri_name)

          except:
            pass

        for i in range(len(datas0)):
          loo_df[datas0[i]]=[str(datas1[i])]

        loo_df['company']=[cmp_name]
        loo_df['Url_number']=[num]
        loo_df['Status']=[stus_code]

        loo_df['Test']=["Allow"]
    except:
       if data_find in "You have exceeded the number accesses allowed.":
        time.sleep(1)
        return harvest_page(num)
       else:
        loo_df['Test']=["Not Have"]

    return loo_df
  except:
    return loo_df

In [16]:
# dtas_oyut=harvest_page('000000002')
harvest_page('000000114')

,Entity ID Number,Entity Type,Principal Address,Principal Mailing Address,Status,Place of Formation,Formation Date,Registered Agent Name,Registered Office Street Address,Registered Office Mailing Address,...,Incorporator Street Address,Incorporator Mailing Address,Report Year,Transaction Date,Registered Agent Changed From,Director/Manager/Organizer Activity,Document Date / Type / Pages,company,Url_number,Test
0,000-000-114,Domestic Corporation,"MUSCLE SHOALS, AL",Not Provided,200,Colbert County,11/19/2010,"SINGH, AVTAR","2526 E AVALON AVEMUSCLE SHOALS, AL 35661","2526 E AVALON AVEMUSCLE SHOALS, AL 35661",...,"2526 E AVALON AVEMUSCLE SHOALS, AL 35661","2526 E AVALON AVEMUSCLE SHOALS, AL 35661",2011 \n\n2012 \n\n2013 \n\n2014 \n\n2015 ...,10/13/2021,"NOT PROVIDED2526 E AVALON AVEMUSCLE SHOALS, AL...","*Removed SINGH, KAMAL JIT",10/13/2021 Articles of Amendment 3 pgs.,Singh G Corporation,000000114,Allow


In [ ]:
# dtas_oyut.to_csv('ayeoirtqie.csv')

In [13]:
str_n="000000000"
range_v=[]
for i in range(1,1000):
  st_v=f"{len(str(i))}"
  num_f=str_n[int(st_v):]+str(i)
  range_v.append(num_f)

In [14]:
len(range_v)

999

### Thread

In [10]:
collect_frame=[]
def compile_run():
    with tqdm(total=len(range_v),desc="harvest") as progress:
        with ThreadPoolExecutor(max_workers=10) as executor:
            for i in executor.map(harvest_page,range_v):
               collect_frame.append(i)
               progress.update()
               time.sleep(.1)
               if len(collect_frame)%5==0:
                time.sleep(.2)
compile_run()

harvest: 100%|██████████| 999/999 [55:17<00:00,  3.32s/it]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
final_df=pd.DataFrame()
for i in collect_frame:
    final_df=pd.concat([final_df,i],axis=0,ignore_index=True)

In [15]:
final_df.groupby('Status').count()

,Entity ID Number,Entity Type,Principal Address,Principal Mailing Address,Place of Formation,Formation Date,Registered Agent Name,Registered Office Street Address,Registered Office Mailing Address,Nature of Business,...,Service of Process,Qualify Date,Cancelled Date,General Partner Name,General Partner Street Address,General Partner Mailing Address,Legal Name Consolidated,Manager Name,Manager Street Address,Manager Mailing Address
Status,,,,,,,,,,,,,,,,,,,,,
200.0,783,783,783,783,783,783,783,780,780,783,...,7,22,1,2,2,2,1,1,1,1


In [13]:
final_df

,Entity ID Number,Entity Type,Principal Address,Principal Mailing Address,Status,Place of Formation,Formation Date,Registered Agent Name,Registered Office Street Address,Registered Office Mailing Address,...,Service of Process,Qualify Date,Cancelled Date,General Partner Name,General Partner Street Address,General Partner Mailing Address,Legal Name Consolidated,Manager Name,Manager Street Address,Manager Mailing Address
0,000-000-001,Domestic Corporation,"BIRMINGHAM, AL",Not Provided,200.0,Jefferson County,01/04/1966,Not Provided,Not Provided,Not Provided,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,000-000-002,Domestic Corporation,"HUNTSVILLE, AL",Not Provided,200.0,Madison County,04/17/1959,Not Provided,Not Provided,Not Provided,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,000-000-003,Domestic Corporation,"MONTGOMERY, AL",Not Provided,200.0,Montgomery County,03/17/1971,Not Provided,Not Provided,Not Provided,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,000-000-004,Domestic Non-Profit Corporation,"TALLADEGA, AL",Not Provided,200.0,Talladega County,01/10/1985,"JENKINS, ANN","206 TINNEY STREETTALLADEGA, AL 35160",Not Provided,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
809,000-000-982,Domestic Corporation,Not Provided,Not Provided,200.0,Madison County,01/24/2011,"STREMBICKE, AMANDA","2604 DEFORD MILL RDOWENS CROSS ROADS, AL 35763","2604 DEFORD MILL RDOWENS CROSS ROADS, AL 35763",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
810,000-000-983,Domestic Corporation,Not Provided,Not Provided,200.0,Madison County,01/21/2011,"TANG, XI G","5850 HWY 53 STE AHARVEST, AL 35749","5850 HWY 53 STE AHARVEST, AL 35749",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
811,000-000-984,Domestic Corporation,Not Provided,"3077 LEEMAN FERRY RD STE H2HUNTSVILLE, AL 35801",200.0,Madison County,01/21/2011,"KENT, SHAWN",Not Provided,Not Provided,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
812,000-000-991,Domestic Corporation,Not Provided,Not Provided,200.0,Madison County,01/18/2011,NOT PROVIDED,"4706 SLALOM RUNOWENS CROSS ROADS, AL 35763","4706 SLALOM RUNOWENS CROSS ROADS, AL 35763",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Async Function

In [15]:
import aiohttp
import asyncio
import nest_asyncio

In [30]:
async def harvest_page(session,num):
        loo_df=pd.DataFrame()
        try:
          headers={ "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,application/signed-exchange;v=b3;q=0.7",
                        "Accept-Encoding":"gzip,deflate,br",
                        "Accept-Language":"en-US,en;q=0.9,ta;q=0.8",
                        "Connection":"keep-alive",
                        "Cookie":"_ga=GA1.1.2085200574.1702026901;_ga_HEFYBZ2QN2=GS1.1.1702285595.5.1.1702285603.0.0.0",
                        "Host":"arc-sos.state.al.us",
                        "Sec-Fetch-Dest":"document",
                        "Sec-Fetch-Mode":"navigate",
                        "Sec-Fetch-Site":"none",
                        "Sec-Fetch-User":"?1",
                        "Upgrade-Insecure-Requests":"1",
                        "User-Agent":"Mozilla/5.0(WindowsNT10.0;Win64;x64)AppleWebKit/537.36(KHTML,likeGecko)Chrome/120.0.0.0Safari/537.36",
                        "sec-ch-ua":"'Not_ABrand';v='8','Chromium';v='120','GoogleChrome';v='120'",
                        "sec-ch-ua-mobile":"?0",
                        "sec-ch-ua-platform":"Windows"
                      }
          url=f'https://arc-sos.state.al.us/cgi/corpdetail.mbr/detail?corp={num}'
          async with session.get(url, headers=headers,timeout=15) as response:
              status_code = response.status
              page_content = await response.text()
              #res=requests.get(url,headers=headers,timeout=40)
              sou=BeautifulSoup(page_content,'lxml')
              data_find=sou.text.strip()
              try:
                  cmp_name=sou.find('thead').find('td',class_="aiSosDetailHead").text.strip()
                  stus_code=res.status_code
                  datas0=[]
                  datas1=[]

                  for i in sou.find('div',attrs={'align':'center'}).find_all('tr'):
                    try:
                      attri_name=i.find('td',class_="aiSosDetailDesc").text.strip()
                      detail_value=i.find('td',class_="aiSosDetailValue").text.strip()

                      datas1.append(detail_value)
                      datas0.append(attri_name)

                    except:
                      pass

                  for i in range(len(datas0)):
                    loo_df[datas0[i]]=[str(datas1[i])]

                  loo_df['company']=[cmp_name]
                  loo_df['Url_number']=[num]
                  loo_df['Status']=[stus_code]

                  loo_df['Test']=["Allow"]
              except:
                if data_find in "You have exceeded the number accesses allowed.":
                  time.sleep(1)
                  return harvest_page(num)
                else:
                  loo_df['Url_number']=[num]
                  loo_df['Test']=["Not Have"]

              return loo_df
        except:
          loo_df['Url_number']=[num]
          loo_df['Test']=["Error"]
          return loo_df

### Async Compile

In [ ]:
collect_datas=[]
async def main(links):
    tasks = []
    async with aiohttp.ClientSession() as session:
        for perma in range_v[:500]:
            tasks.append(harvest_page(session, perma))
        results = []

        for task in asyncio.as_completed(tasks):
            data=await task
            collect_frame.append(data)
            progress.update()
            time.sleep(.2)
            if len(collect_frame)%5==0:
            time.sleep(.2)

In [ ]:
if __name__ == "__main__":
    nest_asyncio.apply()
    asyncio.run(main(range_v))

### Loop Complies

In [31]:
tests_range=range_v[:100]

In [32]:
find_final_data=pd.DataFrame()

In [ ]:
while True:
    collect_frame=[]
    async def main(list_num):
        tasks = []
        async with aiohttp.ClientSession() as session:
            for perma in list_num:
                tasks.append(harvest_page(session, perma))
            results = []
            with tqdm(total=len(list_num),desc="harvest") as progress:
                for task in asyncio.as_completed(tasks):
                    data=await task
                    collect_frame.append(data)
                    time.sleep(.2)
                    progress.update(1)
                    if len(collect_frame)%10==0:
                        time.sleep(2)


    if __name__ == "__main__":
        nest_asyncio.apply()
        asyncio.run(main(tests_range))

    final_df=pd.DataFrame()
    for i in collect_frame:
        final_df=pd.concat([final_df,i],axis=0,ignore_index=True)

    find_df=final_df[final_df['Test']!="Error"]
    tests_range=final_df[final_df['Test']=="Error"]['Url_number'].to_list()

    find_final_data=pd.concat([find_df,find_final_data],axis=0)

    print(f"Total Count : {len(collect_frame)}")
    print(f"Find Count : {len(find_df)}")
    print(f"Not Fount : {len(tests_range)}")

    if len(tests_range)==0:
        print('Break')
        break


In [ ]:
tests=find_final_data[['Url_number','Test']]

In [ ]:
tests[tests['Test']!="Allow"]

,Url_number,Test
1,000000006,Not Have
8,000000007,Not Have
9,000000018,Not Have
1,000000011,Not Have
3,000000012,Not Have
12,000000030,Not Have
13,000000041,Not Have
14,000000017,Not Have
15,000000028,Not Have
23,000000032,Not Have
